In [90]:
import dash
from jupyter_dash import JupyterDash
from dash import Dash, dcc, html, dash_table
from dash.dependencies import Input, Output, State
from dash.exceptions import PreventUpdate

import plotly.express as px
import datetime as dt
import time
import numpy as np
import pandas as pd 
import psycopg2
import psycopg2.extras

In [146]:
tab_style = {
    'borderBottom': '1px solid #d6d6d6',
    'padding': '6px',
    'font-family':'Lucida Grande',
    'font-size': '15px',
    'fontWeight': 'bold',
}

tab_selected_style1 = {
    'borderTop': '1px solid #d6d6d6',
    'borderBottom': '1px solid #d6d6d6',
    'backgroundColor': '#ff0018',
    'color': 'white',
    'padding': '6px',
    'font-family':'Lucida Grande',
    'font-size': '15px'
}

tab_selected_style2 = {
    'borderTop': '1px solid #d6d6d6',
    'borderBottom': '1px solid #d6d6d6',
    'backgroundColor': '#ffa52c',
    'color': 'white',
    'padding': '6px',
    'font-family':'Lucida Grande',
    'font-size': '15px'
}

tab_selected_style3 = {
    'borderTop': '1px solid #d6d6d6',
    'borderBottom': '1px solid #d6d6d6',
    'backgroundColor': '#ffff41',
    'color': 'white',
    'padding': '6px',
    'font-family':'Lucida Grande',
    'font-size': '15px'
}

tab_selected_style4 = {
    'borderTop': '1px solid #d6d6d6',
    'borderBottom': '1px solid #d6d6d6',
    'backgroundColor': '#008018',
    'color': 'white',
    'padding': '6px',
    'font-family':'Lucida Grande',
    'font-size': '15px'
}

tab_selected_style5 = {
    'borderTop': '1px solid #d6d6d6',
    'borderBottom': '1px solid #d6d6d6',
    'backgroundColor': '#0000f9',
    'color': 'white',
    'padding': '6px',
    'font-family':'Lucida Grande',
    'font-size': '15px'
}

tab_selected_style6 = {
    'borderTop': '1px solid #d6d6d6',
    'borderBottom': '1px solid #d6d6d6',
    'backgroundColor': '#86007d',
    'color': 'white',
    'padding': '6px',
    'font-family':'Lucida Grande',
    'font-size': '15px'
}

tab_selected_style7 = {
    'borderTop': '1px solid #d6d6d6',
    'borderBottom': '1px solid #d6d6d6',
    'backgroundColor': '#f6aab7',
    'color': 'white',
    'padding': '6px',
    'font-family':'Lucida Grande',
    'font-size': '15px'
}

header = {
    "font-family": "Lucida Grande",
    "font-size": "25px",
    'text-align':'center',
    "letter-spacing": "0.4px",
    "word-spacing": "2px",
    "color": "#ffffff",
    "font-weight": "700",
    "text-decoration": "none",
    "font-style": "normal",
    "font-variant": "small-caps",
    "text-transform": "none",
    'background': 'linear-gradient(to right, #FF0018 0%, #FFA52C 20%,#FFFF41 40%, #008018 60%, #0000F9 80%, #86007D 100%)'
}

searchBox = {
    'width': '99.3%',
    'height': '100px',
    'margin-top':'10px',
    'background-color':'#dedede',
    'borderWidth': '1px',
    'borderStyle': 'solid',
    'borderRadius': '5px',
    'borderColor': '#dedede',
    'padding': '5px',
    'font-family':'Lucida Grande'
}

style_cell = {
    'text-align': 'left',
    'font-family': 'Lucida Grande',
    'font-size': '10px'
}

tbl_header_style = {
    'text-align': 'left',
    'font-family': 'Lucida Grande',
    'font-size': '12px',
    'font-weight': 'bold',
    'background-color': '#dedede'
}


In [147]:
hostname = 'localhost'
database = 'rbike'
username = 'postgres'
pwd = 'admin'
port_id = 5432
conn= None
cur= None

try:
    conn = psycopg2.connect(
        host=hostname,
        dbname=database,
        user=username,
        password=pwd,
        port=port_id)

    cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)

    cur.execute('SELECT * FROM customer')
    data = cur.fetchall()
    df_customer = pd.DataFrame(data=data, columns=['id','first name','last name','sex','email'])
    #print(df_customer)

    cur.execute('SELECT * FROM bike')
    data = cur.fetchall()
    df_bike = pd.DataFrame(data=data, columns=['bike_id','brand','model','repair_status','order_id','repair_shop_id','supplier_id'])
    #print(df_bike)

    cur.execute('SELECT * FROM order_in')
    data = cur.fetchall()
    df_order = pd.DataFrame(data=data, columns=['order_id','payment','customer_id','giftbox_id'])
    #print(df_order)

    cur.execute('SELECT * FROM employee')
    data = cur.fetchall()
    df_employee = pd.DataFrame(data=data,columns=['employee_id','first_name','last_name','sex','e-mail','department_id'])
    #print(df_employee)

    cur.execute('SELECT * FROM supplier')
    data = cur.fetchall()
    df_supplier = pd.DataFrame(data=data, columns=['supplier_id','supplier_name','contact_person','e-mail'])
    #print(df_supplier)

    cur.execute('SELECT * FROM repairshop')
    data = cur.fetchall()
    df_repair = pd.DataFrame(data=data, columns=['repair_shop_id','repair_shop_name','contact_person','e-mail'])
    #print(df_repair)

    cur.execute('SELECT * FROM giftbox')
    data = cur.fetchall()
    df_gift = pd.DataFrame(data=data, columns=['giftbox_id','giftbox_name','giftbox_procurement_price','giftbox_amount'])
    #print(df_gift)

    #insert_script= 'INSERT INTO customer (first_name, last_name, sex, email) VALUES (%s,%s,%s,%s)'
    #insert_values= ('Mitchell', 'Pritchett','male', 'MVP@modern_family.com')
    #cur.execute(insert_script,insert_values)
    
    conn.commit()

except Exception as error:
    print(error)

finally:
    if cur is not None:
        cur.close()
        conn.close()
    

In [148]:
app = JupyterDash(__name__)
app.layout = html.Div(children=[
    html.H1(children=['RBike Database Management System'], style=header),
    html.Div(id='home', className='control_tabs',children=[
        dcc.Tabs(id='view_tabs',value='bikes', children=[
            dcc.Tab(
                label='Bikes',
                value='bikes',
                style=tab_style,
                selected_style= tab_selected_style1,
                children=[
                    html.Div(style={'margin-top':'10px'},children=[
                        dash_table.DataTable(
                            df_bike.to_dict('records'), [{"name": i, "id": i} for i in df_bike.columns],
                            style_cell=style_cell,
                            style_header=tbl_header_style,
                            style_as_list_view=True,
                            id='bike_tbl'
                        )
                    ])
                ]
            ),
            dcc.Tab(
                label='Customers',
                value='customer',
                style=tab_style,
                selected_style= tab_selected_style2,
                children=[
                    html.Div(style={'margin-top':'10px'},children=[
                        dash_table.DataTable(
                            df_customer.to_dict('records'), [{"name": i, "id": i} for i in df_customer.columns],
                            style_cell=style_cell,
                            style_header=tbl_header_style,
                            style_as_list_view=True,
                            id='customer_tbl'
                        )
                    ])
                ]
            ),
            dcc.Tab(
                label='Bookings',
                value='bookings',
                style=tab_style,
                selected_style=tab_selected_style3,
                children= [
                    html.Div(style= searchBox,children=[
                        html.H5(children=['Search Box'],style={'margin-top':'-2px'}),
                        dcc.Input(placeholder='what are you looking for?', type='text',id='input 1'),
                        dcc.Input(placeholder='what are you looking for?', type='text',id='input 2'),
                        html.P(children='hello',style={'margin-left':'90%'})
                    ]),
                    html.Div(style={'margin-top':'10px'},children=[
                        dash_table.DataTable(
                            df_order.to_dict('records'), [{"name": i, "id": i} for i in df_order.columns],
                            style_cell=style_cell,
                            style_header=tbl_header_style,
                            style_as_list_view=True,
                            id='order_tbl'
                        )
                    ])
                ]
            ),
            dcc.Tab(
                label='H.R.',
                value='hr',
                style=tab_style,
                selected_style=tab_selected_style4,
                children= [
                    html.Div(style={'margin-top':'10px'},children=[
                        dash_table.DataTable(
                            df_employee.to_dict('records'), [{"name": i, "id": i} for i in df_employee.columns],
                            style_cell=style_cell,
                            style_header=tbl_header_style,
                            style_as_list_view=True,
                            id='employee_tbl'
                        )
                    ])
                ]
            ),
            dcc.Tab(
                label='Suppliers',
                value='suppliers',
                style=tab_style,
                selected_style=tab_selected_style5,
                children=[
                    html.Div(style={'margin-top':'10px'},children=[
                        dash_table.DataTable(
                            df_supplier.to_dict('records'), [{"name": i, "id": i} for i in df_supplier.columns],
                            style_cell=style_cell,
                            style_header=tbl_header_style,
                            style_as_list_view=True,
                            id='supplier_tbl'
                        )
                    ])
                ]
            ),
            dcc.Tab(
                label='Repair shops',
                value='repair_shops',
                style=tab_style,
                selected_style=tab_selected_style6,
                children=[
                     html.Div(style={'margin-top':'10px'},children=[
                        dash_table.DataTable(
                            df_repair.to_dict('records'), [{"name": i, "id": i} for i in df_repair.columns],
                            style_cell=style_cell,
                            style_header=tbl_header_style,
                            style_as_list_view=True,
                            id='repair_tbl'
                        )
                    ])
                ]
            ),
            dcc.Tab(
                label='Gift Boxes',
                value='giftbox',
                style=tab_style,
                selected_style=tab_selected_style7,
                children=[
                    html.Div(style={'margin-top':'10px'},children=[
                        dash_table.DataTable(
                            df_gift.to_dict('records'), [{"name": i, "id": i} for i in df_gift.columns],
                            style_cell=style_cell,
                            style_header=tbl_header_style,
                            style_as_list_view=True,
                            id='gift_tbl'
                        )
                    ])
                ]
            )
        ])
    ])
])

In [149]:
if __name__ == '__main__':
    app.run_server(debug=True, mode='external', host='127.0.0.1',port=8000)

Dash app running on http://127.0.0.1:8000/
